In [ ]:
## このプログラムは「CIFAR-10 - Object Recognition in Images」において
## 作成したノートブックで動作します
## ローカル環境のJupyter Notebookで作成したノートブックでも動作可能です

# モデルを生成する
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras        import optimizers

def make_convlayer():
    # Sequentialオブジェクト
    model = Sequential()
    # 畳み込み層1
    model.add(Conv2D(
        filters=64, kernel_size=3, padding='same',
        activation='relu',input_shape=(32,32,3)))
    # 2×2のプーリング層
    model.add(MaxPooling2D(pool_size=2))
    # 畳み込み層2
    model.add(Conv2D(
        filters=128, kernel_size=3, padding='same', 
        activation='relu'))
    # 2×2のプーリング層
    model.add(MaxPooling2D(pool_size=2))
    #畳み込み層3
    model.add(Conv2D(
        filters=256, kernel_size=3, padding='same', 
        activation='relu'))
    #2×2のプーリング層
    model.add(MaxPooling2D(pool_size=2))
    # Flatten層
    model.add(Flatten())
    # ドロップアウト
    model.add(Dropout(0.4))
    # 第7層
    model.add(Dense(512, activation='relu'))
    # 出力層
    model.add(Dense(10, activation = "softmax"))

    # オプティマイザーはAdam
    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=["accuracy"])    
    return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import History, LearningRateScheduler

def step_decay(epoch):
    """1/5ずつ学習率を減衰する
    
    Parameters:epoch(int):エポック数
    Returns   : 学習率
    """
    lrate = 0.001
    if epoch >= 50: lrate /= 5.0
    if epoch >= 100: lrate /= 5.0
    if epoch >= 150: lrate /= 5.0
    return lrate

def train_batchsize(model, data, batch_size, epochs, decay):
    """
    Parameters:
      model(obj)     : Modelオブジェクト
      data(tuple)    : 訓練データ、テストデータ
      batch_size(int): バッチサイズ
      epochs(int)    : エポック数
      decay(float)   : 学習率
    """
    x_train, y_train, x_test, y_test = data
    # 訓練データ
    train_gen = ImageDataGenerator(
        rescale=1.0/255.0,      # 正規化
        width_shift_range=0.1,  # 0.1の割合でランダムに水平移動
        height_shift_range=0.1, # 0.1の割合でランダムに垂直移動
        rotation_range=10,      # 10度の範囲でランダムに回転させる
        zoom_range=0.1,         # ランダムに拡大
        horizontal_flip=True    # 左右反転
        ).flow(x_train, y_train, batch_size=batch_size)
    # テストデータ
    test_gen = ImageDataGenerator(
        rescale=1.0/255.0       # 正規化
        ).flow(x_test, y_test, batch_size=128) # バッチサイズは128

    hist = History()
    # 学習
    model.fit(
        train_gen,
        steps_per_epoch=x_train.shape[0]//batch_size,
        epochs=epochs,
        validation_data=test_gen,
        callbacks=[hist, decay])
    
    return hist.history

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def train(train_mode):
    """
    train_mode(int);
      # 0: normal batch_size=128,
           lr=0.001,0.0002‬, 0.00004‬, 0.000008‬
      # 1: increase batch = 128, 640, 3200, 16000
           lr=0.001
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    data = (x_train, y_train, x_test, y_test)

    # モデル生成
    model = make_convlayer()

    # Historyオブジェクトを保持するリスト
    histories = []
    # 学習率減衰用のスケジューラー
    decay = LearningRateScheduler(step_decay)
    # 可変バッチサイズ用のスケジューラー
    same_lr = LearningRateScheduler(lambda epoch: 0.001)
    # 学習率減衰
    if train_mode == 0:
        histories.append(train_batchsize(
                model, data, batch_size=128, epochs=200, decay=decay))
    # バッチサイズ増加
    if train_mode == 1:
        histories.append(train_batchsize(
                model, data, batch_size=128, epochs=50, decay=same_lr))
        histories.append(train_batchsize(
                model, data, batch_size=640, epochs=50, decay=same_lr))
        histories.append(train_batchsize(
                model, data, batch_size=3200, epochs=50, decay=same_lr))
        histories.append(train_batchsize(
                model, data, batch_size=16000, epochs=50, decay=same_lr))

    # Historyの統合
    joined_history = histories[0]
    for i in range(1, len(histories)):
        for key, value in histories[i].items():
            joined_history[key] = joined_history[key] + value
            
    return joined_history

In [ ]:
%%time
history = train(0)

In [ ]:
# 訓練時の精度の推移をグラフ化
import matplotlib.pyplot as plt
%matplotlib inline

plt.ﬁgure(ﬁgsize=(15, 15)) # プロット図のサイズ
plt.subplot(2, 1, 1)       # 2×1のグリッドの上部にプロット

# 学習率減衰の精度をプロット
plt.plot(
    history['accuracy'], label='lr', color='black')
# バッチサイズ増加の精度をプロット
plt.plot(
    history_batch['accuracy'], label='batch',color='red')
plt.legend()         # 凡例表示
plt.grid()           # グリッド表示
plt.xlabel('Epoch')  # x軸ラベル
plt.ylabel('Acc')    # y軸ラベル

In [ ]:
# 検証データの精度の推移をグラフ化
import matplotlib.pyplot as plt
%matplotlib inline

plt.ﬁgure(ﬁgsize=(15, 15)) # プロット図のサイズ
plt.subplot(2, 1, 1)       # 2×1のグリッドの上部にプロット

# 学習率減衰の精度をプロット
plt.plot(
    history['val_accuracy'], label='lr', color='black')
# バッチサイズ増加の精度をプロット
plt.plot(
    history_batch['val_accuracy'], label='batch',color='red')
plt.legend()         # 凡例表示
plt.grid()           # グリッド表示
plt.xlabel('Epoch')  # x軸ラベル
plt.ylabel('Acc')    # y軸ラベル